There was no way to batch download images off Civitai and that really bugged me so I made this! I can't take all the credit, I did use ChatGPT to do this as coding just makes me want to yeet myself out a building window.

**Nothing gets sent to ChatGPT**, **nothing gets sent to me**, this just saves your Civitai images to your Google Drive. Google's data handling policies and privacy policies obviously apply here.

Below you'll need to hit the "play" or run button on these cells **in order**. The first cell will ask you for 3 things - your Civitai username (this is to ensure it downloads your images and not all the images on the site or someone else's images), your API key (you can access that at http://https://civitai.com/user/account), and the number of images to download (you can find this on your profile and going across the top menu that displays Models, Posts, Images, etc.) if you don't define the number of images then it will just keep downloading until either **Civitai bans you from the API** or **Google stops giving you compute power** so including the number of images you're downloading is important. **This tool cannot download individual images in any kind of predicatble way, this is only for batch downloading all your images at once**.

# **User Inputs (ONLY CHANGE THESE TO SUIT YOUR OWN USERNAME, API KEY, AND IMAGES)**

Click the run button and a text box will become visible to ask you for the following inputs;

In [2]:
# User inputs
civitai_username = input("Enter the Civitai username: ")
api_key = input("Enter your API key: ")
max_images = int(input("Enter the number of images to download: "))

Enter the Civitai username: AilurosSapiens
Enter your API key: b8ff0cb063a3a9871630a2b54557cb3a
Enter the number of images to download: 338


# **Mount Google Drive**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Install Required Libraries**

In [4]:
!pip install requests tenacity tqdm

# **Error Logging**

In [5]:
import logging
from tqdm import tqdm

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# **Define Helper Functions**

In [6]:
import requests
from tenacity import retry, stop_after_attempt, wait_fixed, RetryError

# Retry configuration
@retry(stop=stop_after_attempt(3), wait=wait_fixed(5))
def fetch_url(url, headers, params=None):
    logging.info(f"Fetching URL: {url} with params: {params}")
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response

# **Set Up API and Directory**

In [7]:
import os

# API Endpoint and API key
API_ENDPOINT = "https://civitai.com/api/v1/images"
SAVE_DIR = "/content/drive/MyDrive/Civ Downloads"  # Google Drive directory

# Create directory to save images
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

# **Download Function**

In [8]:
import time

def download_images(civitai_username, api_key, max_images):
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    params = {
        "username": civitai_username  # Using the specified username
    }

    image_counter = 1
    next_page = API_ENDPOINT
    while next_page and image_counter <= max_images:
        try:
            response = fetch_url(next_page, headers, params)
            data = response.json()
            logging.info(f"Fetched page: {next_page}")
            images = data["items"]
            for image in images:
                if image_counter > max_images:
                    break
                image_url = image["url"]
                image_name = f"{civitai_username}_civ({image_counter}).jpg"
                try:
                    image_response = fetch_url(image_url, headers)
                    image_path = os.path.join(SAVE_DIR, image_name)
                    with open(image_path, 'wb') as file:
                        file.write(image_response.content)
                    logging.info(f"Downloaded {image_name}")
                    image_counter += 1
                except Exception as e:
                    logging.error(f"Failed to download {image_name}: {e}")

            next_page = data["metadata"].get("nextPage")
            if next_page:
                # Remove duplicate username parameter in next_page URL
                params = None
            else:
                logging.info("No more pages to fetch.")
                break
            time.sleep(1)  # Rate limiting: sleep for 1 second between page requests
        except RetryError as e:
            logging.error(f"Failed to fetch images after retries: {e}")
            break
        except Exception as e:
            logging.error(f"An error occurred: {e}")
            break





# **Download your Images**

In [9]:
if __name__ == "__main__":
    download_images(civitai_username, api_key, max_images)
